In [63]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [64]:
def variable_init(size):
    in_dim = size[0]
#     variable_stddev = 1./tf.sqrt(in_dim/2.)
    variable_stddev = tf.sqrt(2./in_dim)
    return tf.random_normal(shape=size,stddev=variable_stddev)
#     return tf.random_normal(shape=size)

In [65]:
mb_size = 32
Z_dim = 10

In [66]:
X = tf.placeholder(tf.float32, shape=[None,784])

D_W1 = tf.Variable(variable_init([784,128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(variable_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


In [67]:
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(variable_init([Z_dim,128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(variable_init([128,784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [68]:
def sample_Z(m,n):
    return np.random.uniform(-1., 1., size=[m,n])

In [69]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_prob = tf.nn.sigmoid(tf.matmul(G_h1, G_W2) + G_b2)
    return G_prob

In [70]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    # Remove sigmoid from origin GAN
#     D_prob = tf.nn.sigmoid(D_logit)
    return D_logit

In [71]:
def plot(samples):
    fig = plt.figure(figsize=(4,4))
    gs = gridspec.GridSpec(4,4)
    gs.update(wspace=0.05, hspace=0.05)
    
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28,28),cmap='Greys_r')
    return fig

In [72]:
G_sample = generator(Z)
D_logit_real = discriminator(X)
D_logit_fake = discriminator(G_sample)

In [73]:
# D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
# D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
# D_loss = D_loss_real + D_loss_fake

# G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

# D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
# G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)



In [74]:
D_loss = tf.reduce_mean(D_logit_real) - tf.reduce_mean(D_logit_fake)
G_loss = -tf.reduce_mean(D_logit_fake)

# Here we need to add minus, because the update of w_d is w_d = w_d + lr * descent
# However for tensorflow, its defulat operation should be w = w - lr * descent
D_solver = tf.train.RMSPropOptimizer(learning_rate=1e-4).minimize(-D_loss, var_list=theta_D)
G_solver = tf.train.RMSPropOptimizer(learning_rate=1e-4).minimize(G_loss, var_list=theta_G)

# Only clip the w, no b
clip_D = [p.assign(tf.clip_by_value(p,-0.01,0.01)) for p in theta_D[:2]]


In [75]:
mnist = input_data.read_data_sets('../../common_dataset/MNIST_data',one_hot=True)

Extracting ../../common_dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../common_dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../common_dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../common_dataset/MNIST_data/t10k-labels-idx1-ubyte.gz


In [76]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [77]:
if not os.path.exists('out/'):
    os.makedirs('out/')

In [ ]:
# i=0
# for it in range(10000):
#     if it%1000 == 0:
#         samples = sess.run(G_sample, feed_dict={Z:sample_Z(16,Z_dim)})
        
#         fig = plot(samples)
#         plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
#         i+=1
#         plt.close(fig)
    
#     X_mb, _ = mnist.train.next_batch(mb_size)
#     _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X:X_mb, Z:sample_Z(mb_size,Z_dim)})
#     _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z:sample_Z(mb_size, Z_dim)})
#     if it % 1000 == 0:
#         print('Iter:{}'.format(it))
#         print('D_loss:{:.4}'.format(D_loss_curr))
#         print('G_loss:{:.4}'.format(G_loss_curr))
#         print()

In [ ]:
i = 0
for it in range(10000):
    for _ in range(5):
        X_mb, _ = mnist.train.next_batch(mb_size)
        
        _, D_loss_curr, _ = sess.run([D_solver, D_loss, clip_D],
                                    feed_dict={X:X_mb, Z:sample_Z(mb_size, Z_dim)})
    
    _, G_loss_curr = sess.run([G_solver, G_loss],
                             feed_dict={Z:sample_Z(mb_size, Z_dim)})
    
    if it % 100 == 0:
        print('Iter:{}; D_loss: {:.4}; G_loss: {:.4}'.format(it, D_loss_curr, G_loss_curr))
        
        if it % 1000 == 0:
            samples = sess.run(G_sample, feed_dict={Z:sample_Z(16,Z_dim)})
            fig = plot(samples)
            plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)
    

Iter:0; D_loss: -0.00185; G_loss: -0.004683
Iter:100; D_loss: 1.864; G_loss: 1.482
Iter:200; D_loss: 1.837; G_loss: 1.317
Iter:300; D_loss: 1.551; G_loss: 1.047
Iter:400; D_loss: 1.31; G_loss: 0.8539
Iter:500; D_loss: 1.023; G_loss: 0.7486
Iter:600; D_loss: 0.7314; G_loss: 0.6002
Iter:700; D_loss: 0.4776; G_loss: 0.4923
Iter:800; D_loss: 0.3171; G_loss: 0.1116
Iter:900; D_loss: 0.1153; G_loss: -0.105
Iter:1000; D_loss: 0.06079; G_loss: -0.2318
Iter:1100; D_loss: 0.06331; G_loss: -0.1703
Iter:1200; D_loss: 0.02523; G_loss: -0.1925
Iter:1300; D_loss: 0.01195; G_loss: -0.3216
Iter:1400; D_loss: 0.01575; G_loss: -0.2808
Iter:1500; D_loss: 0.003801; G_loss: -0.2417
Iter:1600; D_loss: 0.009472; G_loss: -0.2177
Iter:1700; D_loss: 0.007447; G_loss: -0.2341
Iter:1800; D_loss: 0.006432; G_loss: -0.2945
Iter:1900; D_loss: 0.01477; G_loss: -0.3487
Iter:2000; D_loss: 0.01318; G_loss: -0.3736
Iter:2100; D_loss: 0.01352; G_loss: -0.1205
Iter:2200; D_loss: 0.003655; G_loss: -0.2891
Iter:2300; D_loss: 